# 🌌 Prototipo: IA para Identificar Estrellas y Detectar Posibles Galaxias

Este notebook guía paso a paso (para principiantes) cómo:
1. Descargar datos reales de **Gaia DR3**
2. Prepararlos para *machine learning*
3. Entrenar un modelo sencillo para clasificar estrellas (azul/roja)
4. Visualizar la estructura de la Vía Láctea (diagrama color–magnitud)
5. Detectar **anomalías** que podrían indicar galaxias o cúmulos

> Requisitos: Python 3.9+, `pip install -r requirements.txt`


In [ ]:
!pip -q install astroquery astropy pandas lightgbm scikit-learn matplotlib

## 1) Descargar datos (Gaia DR3)
Limitamos a 50k filas para mantenerlo ligero.

In [ ]:
from astroquery.gaia import Gaia
import pandas as pd

query = """
SELECT TOP 50000
  source_id, ra, dec, parallax, pmra, pmdec,
  phot_g_mean_mag, bp_rp, ruwe
FROM gaiadr3.gaia_source
WHERE ruwe < 1.4 AND parallax_over_error > 5
"""
print("Lanzando consulta a Gaia...")
job = Gaia.launch_job_async(query)
gaia = job.get_results().to_pandas()
gaia.to_csv("data/estrellas_gaia.csv", index=False)
len(gaia)

## 2) Preparar datos
- Convertimos paralaje a distancia (parsecs)
- Calculamos la magnitud absoluta `MG`
- Eliminamos nulos

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv("data/estrellas_gaia.csv")
data["dist_pc"] = 1000.0 / data["parallax"]
data["MG"] = data["phot_g_mean_mag"] - 5*np.log10(data["dist_pc"]/10)
data = data.dropna(subset=["bp_rp", "MG"])
data.to_csv("data/estrellas_limpias.csv", index=False)
len(data)

## 3) Entrenar IA sencilla (clasificar azul/roja)
Esto es **didáctico**: usamos el color `BP-RP` para crear etiquetas.

In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import pandas as pd

data = pd.read_csv("data/estrellas_limpias.csv")
data["tipo"] = data["bp_rp"].apply(lambda x: "Azul" if x < 0.8 else "Roja")
X = data[["bp_rp", "MG", "parallax", "pmra", "pmdec"]]
y = data["tipo"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

## 4) Visualizar la Vía Láctea (diagrama color–magnitud)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("data/estrellas_limpias.csv")
plt.figure(figsize=(8,6))
plt.scatter(data["bp_rp"], data["MG"], s=1)
plt.gca().invert_yaxis()
plt.xlabel("Color (BP - RP)")
plt.ylabel("Magnitud Absoluta (MG)")
plt.title("Diagrama Color-Magnitud - Vía Láctea (Gaia DR3)")
plt.show()

## 5) Detectar anomalías (posibles galaxias/cúmulos)
Usamos `IsolationForest` para encontrar objetos que no siguen el patrón típico.

In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd
data = pd.read_csv("data/estrellas_limpias.csv")
X = data[["bp_rp", "MG"]]
model = IsolationForest(contamination=0.01, random_state=42)
model.fit(X)
data["anomalia"] = model.predict(X)
anomalias = data[data["anomalia"] == -1]
anomalias.to_csv("data/posibles_galaxias.csv", index=False)
len(anomalias)

### ✅ ¡Listo!
- `data/estrellas_gaia.csv`: datos descargados
- `data/estrellas_limpias.csv`: datos preparados
- `data/posibles_galaxias.csv`: anomalías detectadas

Siguientes pasos (opcional):
- Probar más columnas de Gaia (velocidad radial, RUWE, etc.)
- Mejorar la clasificación (más clases espectrales)
- Unir datos de SDSS/LAMOST para etiquetas reales de espectros
